In [13]:
#import reqd libraries
import numpy as np
import pandas as pd
#os or pathlib for extracting images in different folders
import os
#keras for dealing with images
from keras.preprocessing import image
from pathlib import Path
import matplotlib.pyplot as plt
import random

In [14]:
p=Path("Desktop\images")
#now p is the path to our data set folder named images which contains 4 folders i.e cats, dogs, horses, humans
#glob fn gives the files/folders of type we entered. * means all files/folders
dirs=p.glob("*")
labels_dict={"cat":0,"dog":1,"horse":2,"human":3}
l_dict={0:"cat",1:"dog",2:"horse",3:"human"}
#to stores images and their corresponding labels
image_data=[]
labels=[]

In [15]:
#traverse for folder paths in dirs
for folder_name in dirs:
    label=str(folder_name).split('\\')[-1][:-1]
    #folder_name is of type path so convert it to string and we need only last word i.e for eg cats out of which we remove 's' from all
    #print(label)
    #now traverse in each of these folders. we need only images so use jpg in glob
    for img_path in folder_name.glob("*.jpg"):
        #print(img_path)
        #load the image using keras and fix its size as images maybe of differnent sizes
        img=image.load_img(img_path,target_size=(32,32))
        #convert image to array
        img_array=image.img_to_array(img)
        #append the image to image data
        image_data.append(img_array)
        #append the corresponding label to labels list
        labels.append(labels_dict[label])

In [16]:
#check that length of both image_data and labels is same i.e 808 in our case
#print(len(labels))
#print(len(image_data))
#convert image data to numpy array..
#since we will plot it on matplotlib so all values should be between 0-1 and also convert it to float
image_data=np.array(image_data,dtype='float32')/255.0
labels=np.array(labels)
#print(image_data)
#print(image_data.shape,labels.shape)
#by shape we see that their are 808 examples with 100*100 size and 3 channels
#now shuffle the dataset.. corresponding labels should stay the way they are
#so combine them into one and then shuffle
#to combine we zip, then shuffle and unzip
combined=list(zip(image_data,labels))
random.shuffle(combined)
#unzip
image_data[:],labels[:]=zip(*combined)
#check for some images
#for i in range(10):
    #plt.imshow(image_data[i])
    #does not show axis with image
    #plt.axis('off')
    #plt.title(str(labels[i]))
    #plt.show()


In [17]:
#SVM Code with only 2 classes i.e -1 & 1
class SVM:
    def __init__(self,C=1.0):
        #C->hyperparameter
        self.C = C
        #W-> weights
        self.W = 0
        #b->bias
        self.b = 0
        
    def hingeLoss(self,W,b,X,Y):
        loss  = 0.0
        
        loss += .5*np.dot(W,W.T)
        
        m = X.shape[0]
        
        for i in range(m):
            ti = Y[i]*(np.dot(W,X[i].T)+b)
            loss += self.C *max(0,(1-ti))
            
        return loss[0][0]
    
    def fit(self,X,Y,batch_size=100,learning_rate=0.00001,maxItr=1000):
        
        no_of_features = X.shape[1]
        no_of_samples = X.shape[0]
        
        n = learning_rate
        c = self.C
        
        #Init the model parameters
        W = np.zeros((1,no_of_features))
        bias = 0
        
        #Initial Loss
        
        #Training from here...
        # Weight and Bias update rule!
        losses = []
        
        for i in range(maxItr):
            #Training Loop
            
            l = self.hingeLoss(W,bias,X,Y)
            losses.append(l)
            ids = np.arange(no_of_samples)
            np.random.shuffle(ids)
            
            #Batch Gradient Descent(Paper) with random shuffling
            for batch_start in range(0,no_of_samples,batch_size):
                #Assume 0 gradient for the batch
                gradw = 0
                gradb = 0
                
                #Iterate over all examples in the mini batch
                for j in range(batch_start,batch_start+batch_size):
                    if j<no_of_samples:
                        i = ids[j]
                        ti =  Y[i]*(np.dot(W,X[i].T)+bias)
                        
                        if ti>1:
                            gradw += 0
                            gradb += 0
                        else:
                            gradw += c*Y[i]*X[i]
                            gradb += c*Y[i]
                            
                #Gradient for the batch is ready! Update W,B
                W = W - n*W + n*gradw
                bias = bias + n*gradb
                
        
        self.W = W
        self.b = bias
        return W,bias,losses

In [18]:
#Since we are using a binary classifier! So convert data for One-Vs-One Classifier
#So we will make a data dictionary... in which we will map class label with its data
n_samples=image_data.shape[0]
#flatten the image data to store it in the form of a single row 
image_data=image_data.reshape((n_samples,-1))
#now shape is 808,3072
#print(image_data.shape)

In [19]:
#define a fn to create data classwise i.e all cat examples with cat label, all dog pics with dog label
def classWiseData(image_data,labels):
    #initialise an empty dictionary
    data={}
    #number of classes
    classes=len(np.unique(labels))
    #initialise each label in dictionary
    for i in range(classes):
        data[i]=[]
    for i in range(labels.shape[0]):
        data[labels[i]].append(image_data[i])
    #now in data dictionary we have labels mapped to list of numpy arrays 
    #convert these lists to numpy array
    for j in data.keys():
        data[j]=np.array(data[j])
    return data

In [20]:
data=classWiseData(image_data,labels)

In [21]:
#this function will take data of 2 labels and  prepare the data for svm i.e marks one label as 1 & other as -1 i.e Y  and combine features to get X
#it combines data of two classes into single matrix
def GetDataPairForSVM(data_1,data_2):
    #no of samples in each label
    l1,l2=data_1.shape[0],data_2.shape[0]
    #total no of samples
    n_samples=l1+l2
    n_features=data_1.shape[1]
    #here data_pair is X & data_lables is Y
    data_pair=np.zeros((n_samples,n_features))
    data_labels=np.zeros((n_samples,))
    #copy the data from data_1 & data_2
    data_pair[:l1,:]=data_1
    data_pair[l1:,:]=data_2
    #data_1 class labelled as -1 & data_2 class labelled as 1
    data_labels[:l1]=-1
    data_labels[l1:]=1
    return data_pair,data_labels

In [22]:
#Training Nc2 number of svm's for one vs one classification
mysvm=SVM()
#instead of making nc2 svm's we use a single svm object, train it , store its wts and bias, then again train it with some other pair of data

In [23]:
#it will return a dictionary of weights and biases
def trainSVMs(data,labels):
    #initailise an empty dict
    svm_classifiers={}
    #number of classes
    n_classes=len(np.unique(labels))
    #to go to each pair we use loop i->n & j from i+1->n
    for i in range(n_classes):
        #initialise an empty dictionary for i as label 1(isme j change hota rahega)
        svm_classifiers[i]={}
        for j in range(i+1,n_classes):
            #now we take data of label i and label j and make it suitable for SVM
            x,y=GetDataPairForSVM(data[i],data[j])
            #train the model using this data
            wts,b,loss=mysvm.fit(x,y)
            #store the weights and bias in this dictionary as a tuple
            svm_classifiers[i][j]=(wts,b)
    #return this dictionary of weights and biases
    return svm_classifiers

In [24]:
#train the SVM's
svm_classifiers=trainSVMs(data,labels)

In [25]:
#parameters for cats & dogs
#cats_dogs=svm_classifiers[0][1]
#wts=cats_dogs[0]
#bias=cats_dogs[1]
classes=len(np.unique(labels))
def binaryPredict(x,w,b):
    z=np.dot(x,w.T)+b
    if z>=0:
        return 1
    else :
        return -1
def predict(X):
    count=np.zeros((classes,))
    #we will count ki each classifier given testing point ko kaunsi class me batata hai and then take the maxm of those
    for i in range(classes):
        for j in range(i+1,classes):
            w,b=svm_classifiers[i][j]
            #Take a majority prediction from each of classifier
            #now take its prediction from svm. it will return 1 or -1
            #we know that we kept 1 for j and -1 for i( see getpairdataforsvm function)
            z=binaryPredict(X,w,b)
            if z==1 :
                count[j]+=1
                #increment count of j if prediction is in its favour
            else :
                count[i]+=1
    final_prediction=np.argmax(count)
    return final_prediction

count=0
for i in range(image_data.shape[0]):
    if(predict(image_data[i])==labels[i]):
        count+=1
accuracy=count/image_data.shape[0]
print(accuracy)

0.6113861386138614
